In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from itertools import *
df = pd.read_csv("x_train.csv")
test = pd.read_csv("x_test.csv")
y_train = pd.read_csv("y_train.csv")
sample = pd.read_csv("sample_submission.csv")
pd.set_option('display.max_rows', 20)
print(df.shape)
df

(676, 70)


,id,cat,cow,dog,horse,sheep,bounds_x,bounds_y,bounds_width,bounds_height,...,L_F_paw_y,R_F_paw_v,R_F_paw_x,R_F_paw_y,L_B_paw_v,L_B_paw_x,L_B_paw_y,R_B_paw_v,R_B_paw_x,R_B_paw_y
0,265,0,1,0,0,0,37,16,240,170,...,167,1,161,168,1,94,173,1,55,177
1,676,0,0,0,1,0,43,8,208,236,...,213,1,193,209,1,77,232,1,115,233
2,441,0,0,1,0,0,65,0,234,222,...,0,0,0,0,0,0,0,0,0,0
3,328,0,1,0,0,0,15,8,272,178,...,171,1,190,172,1,52,176,1,35,178
4,102,1,0,0,0,0,2,1,148,146,...,130,1,68,131,1,3,128,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,356,0,1,0,0,0,15,1,266,188,...,0,0,0,0,0,0,0,0,0,0
672,960,0,0,0,0,1,123,72,702,580,...,615,1,618,617,1,172,599,1,223,609
673,812,0,0,0,0,1,57,12,176,176,...,170,1,171,175,1,80,172,1,107,176
674,132,1,0,0,0,0,19,8,278,214,...,218,1,130,219,1,290,106,1,260,121


In [23]:
df.isna().sum().sum()

0

In [24]:
ID_df = df['id']
ID = test['id']
df = df.drop(columns=['id'])
test = test.drop(columns=['id'])

In [25]:
y_train = y_train.drop(columns=['id'])
mapping = {'nothing': 0, 'stand': 1, 'sit': 2, 'lie': 3, 'go': 4, 'run': 5}
y_train = y_train.idxmax(axis=1).map(mapping)
y_train


0      4
1      4
2      0
3      1
4      3
      ..
671    1
672    1
673    1
674    3
675    0
Length: 676, dtype: int64

In [26]:
corr_matrix = df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] >= 0.9)]
df = df.drop(columns=to_drop)
test = test.drop(columns=to_drop)


In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

clf = DecisionTreeClassifier()

param_grid = {
    'max_depth': [4, 5, 6, 7, 8, 9, 10],
    'min_samples_split': [2, 3, 4, 5],
    'min_samples_leaf': [1, 2, 3, 4, 5]
}
X_train, X_test, Y_train, Y_test = train_test_split(df, y_train, test_size=0.3)
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='f1_micro')
grid_search.fit(X_train, Y_train)

print(f'Best parameters: {grid_search.best_params_}')

y_pred_ans = grid_search.predict(test)

y_pred = grid_search.predict(X_test)
f1 = f1_score(Y_test, y_pred, average='micro')
print(f'F1-score: {f1}')


Best parameters: {'max_depth': 6, 'min_samples_leaf': 3, 'min_samples_split': 2}
F1-score: 0.6108374384236454


0.5 скор

In [28]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

param_grid = {
    'max_depth': [5, 15, 25, 35, 45, 55, 65, 75, 85],
    'min_samples_split': [2, 3, 4, 5],
    'min_samples_leaf': [1, 3, 5]
}
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='f1_micro')
grid_search.fit(X_train, Y_train)
print(f'Best parameters: {grid_search.best_params_}')

y_pred_ans = grid_search.predict(test)


y_pred = grid_search.predict(X_test)
f1 = f1_score(Y_test, y_pred, average='micro')
print(f'F1-score micro: {f1}')


Best parameters: {'max_depth': 45, 'min_samples_leaf': 1, 'min_samples_split': 5}
F1-score micro: 0.7044334975369458


0.53448


In [31]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()

param_grid = {
    'max_depth': [3, 5, 7],
    'min_samples_leaf': [1, 3]
}

grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='f1_micro')
grid_search.fit(X_train, Y_train)

print(f'Best parameters: {grid_search.best_params_}')
y_pred_ans = grid_search.predict(test)
y_pred = grid_search.predict(X_test)
f1 = f1_score(Y_test, y_pred, average='micro')
print(f'F1-score micro: {f1}')


Best parameters: {'max_depth': 3, 'min_samples_leaf': 1}
F1-score micro: 0.6059113300492611


0.586

In [32]:
ans = pd.DataFrame({'id': ID, 'label': y_pred_ans})
ans.to_csv('ans.csv', index=False)